In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  ### loading all enviornment variables

In [ ]:
### Access the Chat Model from Groq API

from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi my name is Ankur. I deliver trainings on Gen AI")])

In [ ]:
from langchain_core.messages import AIMessage

In [ ]:
model.invoke([HumanMessage(content="Hi my name is Ankur. I deliver trainings on Gen AI"),
              AIMessage(content="Hi Ankur,\n\nIt's great to meet you!  \n\nI understand you deliver trainings on Gen AI. That's a really exciting field! \n\nWhat kind of trainings do you offer? Do you focus on a specific aspect of Gen AI, like text generation, image creation, or something else?  \n\nI'm always interested in learning more about how people are using AI, so I'd love to hear more about your work.\n"),
              HumanMessage(content="Whats my name and what do I do?")])

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}   ### it keeps a track of sessions between user and model

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:     ### New session to be created if this condition is true
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [ ]:
### Logic to create session ID
config = {"configurable" : {"session_id" : "chat1"}}

In [ ]:
response = with_message_history.invoke([HumanMessage(content="Hi, My name is Ankur and I deliver Gen AI trainings.")] , config = config)

In [ ]:
response.content

In [ ]:
store

In [ ]:
with_message_history.invoke([HumanMessage(content="Whats my name and what I do?")], config = config)

In [ ]:
### Change the config ---> session id

config1 = {"configurable" : {"session_id" : "chat2"}}
response = with_message_history.invoke([HumanMessage(content="Whats my name and what I do?")] , config = config1)
response.content

In [ ]:
response = with_message_history.invoke([HumanMessage(content="My name is Jhon")],
                            config=config1)
response.content

In [ ]:
response = with_message_history.invoke([HumanMessage(content="Whats my name and what I do?")] , config = config1)
response.content

In [ ]:
store

#### Alternative approach by using Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [("system" , "You are a helpful assistant. Answer all the questions to the best of your ability"),
     MessagesPlaceholder(variable_name="messages")]
    )

In [ ]:
chain = prompt | model

In [ ]:
chain.invoke({"messages" : [HumanMessage(content = "My name is Ankr")]})

In [ ]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config3 = {"configurable" : {"session_id" : "chat3"}}
response = with_message_history.invoke([HumanMessage(content="Whats my name?")] , config = config3)
response.content

In [ ]:
store

In [ ]:
response = with_message_history.invoke([HumanMessage(content="My name is Arun")] , config = config3)
response.content

In [ ]:
response = with_message_history.invoke([HumanMessage(content="Whats my name?")] , config = config3)
response.content

In [ ]:
store

In [ ]:
print(store['chat3'])

In [ ]:
response = with_message_history.invoke([HumanMessage(content="Can you summarize the previous conversations in 15 words")] , config = config3)
response.content

### Adding some more features

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [("system" , "You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
     MessagesPlaceholder(variable_name="messages")]
    )

chain = prompt | model

In [ ]:
response = chain.invoke({"messages" : [HumanMessage(content = "My name is Ajay")], "language" : "Hindi"})
response.content

##### We will wrap the above logic in Message History class becuase there are multiple keys/place holders in input. We need to specify the correct key to use to save the chat history

In [ ]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [ ]:
config = {"configurable" : {"session_id" : "chat4"}}
response = with_message_history.invoke({"messages" : [HumanMessage(content="My name is Piyush")] , 
                                        "language" : "Spanish"},
                                        config = config)
response.content

In [ ]:
store

### Managing the Conversation History 

In [ ]:
from langchain_core.messages import trim_messages, SystemMessage

trimmer = trim_messages(max_tokens = 10, strategy = "last", token_counter = model)

In [ ]:
messages = [
    SystemMessage(content = "You are a good assistant"),
    HumanMessage(content="Hi, I am Bob"),
    AIMessage(content="Hi Bob"),
    HumanMessage(content="I like vanilla icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="How are you?"),
    AIMessage(content="I am fine")
]

trimmer.invoke(messages)

In [ ]:
help(trim_messages)